# Deep riccy project

## Env creation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

! tar -zxvf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
!pip install rouge-metric

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

Streaming output truncated to the last 5000 lines.
./refcocog/images/COCO_train2014_000000272463.jpg
./refcocog/images/COCO_train2014_000000485426.jpg
./refcocog/images/COCO_train2014_000000002774.jpg
./refcocog/images/COCO_train2014_000000346160.jpg
./refcocog/images/COCO_train2014_000000199829.jpg
./refcocog/images/COCO_train2014_000000210057.jpg
./refcocog/images/COCO_train2014_000000544117.jpg
./refcocog/images/COCO_train2014_000000440673.jpg
./refcocog/images/COCO_train2014_000000006151.jpg
./refcocog/images/COCO_train2014_000000514391.jpg
./refcocog/images/COCO_train2014_000000247459.jpg
./refcocog/images/COCO_train2014_000000141257.jpg
./refcocog/images/COCO_train2014_000000378896.jpg
./refcocog/images/COCO_train2014_000000514558.jpg
./refcocog/images/COCO_train2014_000000244416.jpg
./refcocog/images/COCO_train2014_000000558486.jpg
./refcocog/images/COCO_train2014_000000377467.jpg
./refcocog/images/COCO_train2014_000000114657.jpg
./refcocog/images/COCO_train2014_000000252905.jpg

In [2]:
#imports

import json
import pickle
import torch
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import clip
import numpy as np

import os
import skimage
import IPython.display

from collections import OrderedDict
import torch
from sklearn.metrics import jaccard_score

## Baseline

### Utilities

In [3]:
def split_string(string):
    string = string.split("_")
    string = string[:-1]
    string = "_".join(string)
    append = ".jpg"
    string = string + append
    
    return string

In [4]:
#define evaluation metrics for bounfing boxes
def iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    if interArea == 0:
        return 0
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

#define jacard index for images that are not the same dimensions
def jacard_index(image1, image2):
    image1 = image1.resize((image2.size[0], image2.size[1]))
    image1 = np.array(image1)
    image2 = np.array(image2)
    image1 = image1.flatten()
    image2 = image2.flatten()
    return jaccard_score(image1, image2, average='macro')


In [5]:
def convert_bbox(bbox, img):
  x1, y1, width, height = bbox
  x2, y2 = x1 + width, y1 + height

  # Verify coordinates
  if x1 < 0 or y1 < 0 or x2 > img.width or y2 > img.height:
      print("Bounding box fuori dai limiti dell'immagine!")
  else:
    return x1, y1, x2, y2

### Metrics

In [6]:
#define evaluation metrics for the model as a class that can be updated on each iteration
class Metrics:
    def __init__(self):
        self.jaccard = []
        self.iou = []

    def update(self, bbox_y, bbox_gt, img1, img2):
        self.jaccard.append(jacard_index(img1, img2))
        self.iou.append(iou(bbox_y, bbox_gt))

    def get(self):
      return np.mean(self.jaccard), np.mean(self.iou)

    def __str__(self):
      return "Jaccard: " + str(np.mean(self.jaccard)) + " IoU: " + str(np.mean(self.iou))

    def reset(self):
      self.jaccard = []
      self.iou = []


### Dataset

In [7]:
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        #load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.ann_frame = pd.DataFrame(data['annotations'])
            self.ann_frame = self.ann_frame.reset_index(drop=False)


        #load annotations
        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        #separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sentences')

        self.refs_frame = self.refs_frame.reset_index(drop=False)

        self.size = self.refs_frame.shape[0]

        self.dataset = pd.merge(self.refs_frame, self.ann_frame, left_on='ann_id', right_on='id')
        self.dataset = self.dataset.drop(columns=['segmentation', 'id', 'category_id_y','ref_id', 'index_x', 'iscrowd', 'image_id_y', 'image_id_x', 'category_id_x', 'ann_id', 'sent_ids', 'index_y', 'area'])

    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        return self.dataset.iloc[idx]

    def get_annotation(self, idx):
      return self.ann_frame.iloc[idx]
    
    def get_imgframe(self, idx):
      return self.img_frame.iloc[idx]

    def get_validation(self):
        #return the dataframe that has as attribute the validation
        return self.dataset[self.dataset['split'] == 'val']
    
    def get_test(self):
        #return the dataframe that has as attribute the test
        return self.dataset[self.dataset['split'] == 'test']
    
    def get_train(self):
        #return the dataframe that has as attribute the train
        return self.dataset[self.dataset['split'] == 'train']


In [8]:
#test dataset

dataset = Coco(local_annotations + 'instances.json', local_annotations + "refs(umd).p")

print(len(dataset))

95010


In [9]:
dataset.dataset

,split,sentences,file_name,bbox
0,test,"{'tokens': ['the', 'man', 'in', 'yellow', 'coa...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
1,test,"{'tokens': ['skiier', 'in', 'red', 'pants'], '...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
2,test,"{'tokens': ['there', 'is', 'red', 'colored', '...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
3,test,"{'tokens': ['a', 'shiny', 'red', 'vintage', 'p...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
4,test,"{'tokens': ['a', 'apple', 'desktop', 'computer...",COCO_train2014_000000478885_124383.jpg,"[338.8, 82.19, 147.34, 157.37]"
...,...,...,...,...
95005,train,"{'tokens': ['the', 'larger', 'banana', 'is', '...",COCO_train2014_000000003518_1042682.jpg,"[141.7, 20.68, 359.02, 144.57]"
95006,train,"{'tokens': ['a', 'guy', 'in', 'black', 'jacket...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95007,train,"{'tokens': ['the', 'man', 'in', 'the', 'dark',...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95008,train,"{'tokens': ['a', 'person', 'in', 'red', 'dress...",COCO_train2014_000000573297_472971.jpg,"[31.71, 67.03, 352.29, 572.97]"


### Model

In [10]:
class VisualGrounding(torch.nn.Module):
    def __init__(self, yolo_version, clip_version, local_path, img_path):
        super(VisualGrounding, self).__init__()
        self.local_path = local_path
        self.img_path = img_path
        #initialize models
        self.yolo = torch.hub.load('ultralytics/yolov5', yolo_version , pretrained=True)
        self.clip, self.preprocess = clip.load(clip_version)
        
        #define metrics
        self.metrics = Metrics()
        
    
    def forward(self, img_path, sentence):
        max_similarity = 0
        max_image = None
        max_bbox = None

        yolo_output = self.yolo(self.local_path+img_path)


        original_img = Image.open(self.local_path+img_path).convert("RGB")

        for i in range(len(yolo_output.xyxy[0])):
            x1 = yolo_output.xyxy[0][i][0].numpy()
            x1 = np.rint(x1)
            y1 = yolo_output.xyxy[0][i][1].numpy()
            y1 = np.rint(y1)
            x2 = yolo_output.xyxy[0][i][2].numpy()
            x2 = np.rint(x2)
            y2 = yolo_output.xyxy[0][i][3].numpy()
            y2 = np.rint(y2)

            img_cropped = original_img.crop((x1, y1, x2, y2))

            img = self.preprocess(img_cropped).unsqueeze(0)

            image_imput = torch.tensor(img)
            
            text = clip.tokenize([sentence])

            with torch.no_grad():
                image_features = self.clip.encode_image(img).float()
                text_features = self.clip.encode_text(text).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

            if similarity > max_similarity:
                max_similarity = similarity
                max_image = img_cropped
                max_bbox = yolo_output.xyxy[0][i]

            return max_bbox, max_image
        
    def evaluate(self, img_path, sentence, gt, original_img):
      bbox = convert_bbox(gt, original_img)
      gt_crop = original_img.crop(bbox)
      prediction_bbox, prediction_img = self.forward(img_path, sentence)
      self.metrics.update(prediction_bbox, bbox, prediction_img, gt_crop)
      return prediction_bbox, prediction_img

    def get_metrics(self):
      return self.metrics.get()
    
    def reset_metrics(self):
      self.metrics.reset()
          
        

### Model evaluation

In [11]:
class VisualGrounding(torch.nn.Module):
    def __init__(self, yolo_version, clip_version, local_path, img_path):
        super(VisualGrounding, self).__init__()
        self.local_path = local_path
        self.img_path = img_path
        #initialize models
        self.yolo = torch.hub.load('ultralytics/yolov5', yolo_version , pretrained=True)
        self.clip, self.preprocess = clip.load(clip_version)
        
        #define metrics
        self.metrics = Metrics()
        
    
    def forward(self, img_path, sentence):
        max_similarity = 0
        max_image = None
        max_bbox = None

        yolo_output = self.yolo(self.local_path+img_path)


        original_img = Image.open(self.local_path+img_path).convert("RGB")

        for i in range(len(yolo_output.xyxy[0])):
            x1 = yolo_output.xyxy[0][i][0].numpy()
            x1 = np.rint(x1)
            y1 = yolo_output.xyxy[0][i][1].numpy()
            y1 = np.rint(y1)
            x2 = yolo_output.xyxy[0][i][2].numpy()
            x2 = np.rint(x2)
            y2 = yolo_output.xyxy[0][i][3].numpy()
            y2 = np.rint(y2)

            img_cropped = original_img.crop((x1, y1, x2, y2))

            img = self.preprocess(img_cropped).unsqueeze(0)

            image_imput = torch.tensor(img)
            
            text = clip.tokenize([sentence])

            with torch.no_grad():
                image_features = self.clip.encode_image(img).float()
                text_features = self.clip.encode_text(text).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

            if similarity > max_similarity:
                max_similarity = similarity
                max_image = img_cropped
                max_bbox = yolo_output.xyxy[0][i]

            return max_bbox, max_image
        
    def evaluate(self, img_path, sentence, gt, original_img):
      bbox = convert_bbox(gt, original_img)
      gt_crop = original_img.crop(bbox)
      prediction_bbox, prediction_img = self.forward(img_path, sentence)
      self.metrics.update(prediction_bbox, bbox, prediction_img, gt_crop)
      return prediction_bbox, prediction_img

    def get_metrics(self):
      return self.metrics.get()
    
    def reset_metrics(self):
      self.metrics.reset()
          
        